# Part I. ETL Pipeline for Pre-Processing the Files

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

file = 'event_datafile_new.csv'

with open(file, 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in the csv file
with open(file, 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra modeling

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Connect to a Cassandra instance on your local machine 
from cassandra.cluster import Cluster
cluster = Cluster()

try:
    # Create cluster object
    cluster = Cluster(['127.0.0.1'])
    # Create a session to connect to the cluster and begin executing queries
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. With Apache Cassandra we want to model the database tables on the queries we want to run.

## The following three questions need to be answered:

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1
- This query must be modeled so that the columns *artist*, *song title* and *song's length* are selected by applying  sessionId and itemInSession as filters. These two in combination will create unique rows therefore we will use them together as a composite primary key.

In [8]:
create_table_query1 = """
CREATE TABLE IF NOT EXISTS played_song(
session_id INT
, session_item INT
, artist_name TEXT
, song_title TEXT
, song_length FLOAT
, PRIMARY KEY(session_id,session_item)
)
"""

try:
    session.execute(create_table_query1)
except Exception as e:
    print(e)

In [9]:
insert_query1 = """
INSERT INTO played_song(
session_id
, session_item
, artist_name
, song_title
, song_length)
VALUES (%s,%s,%s,%s,%s)
"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Identify the corresponding columns by line indexes
        session_id = int(line[8])
        session_item = int(line[3])
        artist_name = line[0]
        song_title = line[9]
        song_length = float(line[5])
        
        # Execute the insert query
        try:
            session.execute(insert_query1, (session_id,session_item,artist_name,song_title,song_length))
        except Exception as e:
            print(e)
            break

In [10]:
select_query1 = """
SELECT artist_name, song_title, song_length FROM played_song 
WHERE session_id = 338 AND session_item = 4
"""

try:
    rows = session.execute(select_query1)
    for row in rows:
        print(row.artist_name, row.song_title, row.song_length)
except Exception as e:
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2
- This query must be modeled so that the *name of the user* and all *songs* and *artists* played in a session are selected in the order in which the songs were played by applying userId and sessionId as filters. We will use userId, sessionId and  sessionItem as primary key which will be composed of userId and sessionId as a composite partition key to make sure all session items are stored in one node for fast data retrieval, and sessionItem as clustering column so that when selecting the songs they appear in the order in which they were played.

In [11]:
create_table_query2 = """
CREATE TABLE IF NOT EXISTS session_songs(
user_id INT
, session_id INT
, session_item INT
, first_name TEXT
, last_name TEXT
, artist_name TEXT
, song_title TEXT
, PRIMARY KEY((user_id,session_id),session_item)
)
"""

try:
    session.execute(create_table_query2)
except Exception as e:
    print(e)

In [12]:
insert_query2 = """
INSERT INTO session_songs(
user_id
, session_id
, session_item
, first_name
, last_name
, artist_name
, song_title
)
VALUES (%s,%s,%s,%s,%s,%s,%s)
"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Identify the corresponding columns by line indexes
        user_id = int(line[10])
        session_id = int(line[8])
        session_item = int(line[3])
        first_name = line[1]
        last_name = line[4]
        artist_name = line[0]
        song_title = line[9]
        
        # Execute the insert query
        try:
            session.execute(insert_query2, (user_id, session_id, session_item, first_name, last_name, artist_name, song_title))
        except Exception as e:
            print(e)
            break

In [13]:
select_query2 = """
SELECT user_id, session_id, session_item, first_name, last_name, artist_name, song_title FROM session_songs 
WHERE user_id = 10 AND session_id = 182
"""

try:
    rows = session.execute(select_query2)
    for row in rows:
        print(row.first_name, row.last_name
              , row.artist_name, row.song_title, row.session_item)
except Exception as e:
    print(e)

Sylvie Cruz Down To The Bone Keep On Keepin' On 0
Sylvie Cruz Three Drives Greece 2000 1
Sylvie Cruz Sebastien Tellier Kilometer 2
Sylvie Cruz Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3


### Query 3
- This query must be modeled so that the *names of all users* who listened to a song are selected by searching the song title. We will use the song title and userId as composite primary key. The song title will be the partition key to make sure all data relating to one song is stored in one node. userId will be the clustering column because it will always be present and unique (unlike the last name and first name).

In [14]:
create_table_query3 = """
CREATE TABLE IF NOT EXISTS song_listeners(
song_title TEXT
, user_id INT
, first_name TEXT
, last_name TEXT
, PRIMARY KEY(song_title,user_id)
)
"""

try:
    session.execute(create_table_query3)
except Exception as e:
    print(e)

In [ ]:
insert_query3 = """
INSERT INTO song_listeners(
song_title
, user_id
, first_name
, last_name
)
VALUES (%s,%s,%s,%s)
"""


file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Identify the corresponding columns by line indexes
        song_title = line[9]
        user_id = int(line[10])
        first_name = line[1]
        last_name = line[4]
        
        # Execute the insert query
        try:
            session.execute(insert_query3, (song_title, user_id, first_name, last_name))
        except Exception as e:
            print(e)
            break

In [ ]:
select_query3 = """
SELECT first_name, last_name FROM song_listeners 
WHERE song_title = 'All Hands Against His Own'
"""

try:
    rows = session.execute(select_query3)
    for row in rows:
        print(row.first_name, row.last_name)
except Exception as e:
    print(e)

## 4. DROP TABLES

In [ ]:
drop_sessionid_sessionitem = "DROP TABLE IF EXISTS sessionid_sessionitem"
drop_userid_sessionid = "DROP TABLE IF EXISTS userid_sessionid"
drop_song_listeners = "DROP TABLE IF EXISTS song_listeners"

drop_tables = [drop_sessionid_sessionitem,drop_userid_sessionid,drop_song_listeners]

for query in drop_tables:
    session.execute(query)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()